In [15]:
from mlflow.tracking import MlflowClient
import pandas as pd
import mlflow
from pathlib import Path

EXPERIMENT_NAME = "sentiment_tweets_clean"
client = MlflowClient()

exp = client.get_experiment_by_name(EXPERIMENT_NAME)
assert exp is not None, "Experiment not found"

runs = client.search_runs([exp.experiment_id], order_by=["metrics.val_auc DESC"], max_results=50)
len(runs), runs[0].data.tags.get("mlflow.runName")


(4, 'bert_distilbert-base-uncased_len96_freeze0')

In [16]:
def run_row(r):
    name = r.data.tags.get("mlflow.runName","")
    m = r.data.metrics
    p = r.data.params
    return {
        "run_name": name,
        "model_name": p.get("model_name", "unknown"),
        "val_auc": m.get("val_auc"),
        "val_f1": m.get("val_f1"),
        "val_precision": m.get("val_precision"),
        "val_recall": m.get("val_recall"),
        "val_accuracy": m.get("val_accuracy"),
        "test_auc": m.get("test_auc"),
        "test_f1": m.get("test_f1"),
        "test_precision": m.get("test_precision"),
        "test_recall": m.get("test_recall"),
        "test_accuracy": m.get("test_accuracy"),
        "fit_time_sec": m.get("fit_time_sec"),
    }

df_runs = pd.DataFrame([run_row(r) for r in runs])
df_runs.sort_values("val_auc", ascending=False).head(10)


,run_name,model_name,val_auc,val_f1,val_precision,val_recall,val_accuracy,test_auc,test_f1,test_precision,test_recall,test_accuracy,fit_time_sec
0,bert_distilbert-base-uncased_len96_freeze0,bert,0.929326,0.854162,0.822426,0.888445,0.848309,0.929083,0.854473,0.823177,0.888244,0.848722,1616.236809
1,dl_lstm_emb100_mw100000_len25,lstm,0.902503,0.824837,0.805947,0.844633,0.820633,0.902447,0.823852,0.806562,0.841900,0.819994,68.362497
2,baseline_tfidf_logreg,unknown,0.898209,0.817130,0.825206,0.809211,0.818902,0.898419,0.816596,0.825789,0.807606,0.818616,25.195124
3,dl_simple_emb100_mw100000_len25,simple,0.882484,0.805125,0.785767,0.825461,0.800203,0.883033,0.805289,0.786811,0.824656,0.800606,62.998503


In [17]:
# exemples de filtres : adapte selon tes run_names
pick = df_runs[
    df_runs["run_name"].str.contains("baseline|dl_lstm|bert_", case=False, na=False)
].copy()

pick = pick.sort_values("val_auc", ascending=False)
pick


,run_name,model_name,val_auc,val_f1,val_precision,val_recall,val_accuracy,test_auc,test_f1,test_precision,test_recall,test_accuracy,fit_time_sec
0,bert_distilbert-base-uncased_len96_freeze0,bert,0.929326,0.854162,0.822426,0.888445,0.848309,0.929083,0.854473,0.823177,0.888244,0.848722,1616.236809
1,dl_lstm_emb100_mw100000_len25,lstm,0.902503,0.824837,0.805947,0.844633,0.820633,0.902447,0.823852,0.806562,0.841900,0.819994,68.362497
2,baseline_tfidf_logreg,unknown,0.898209,0.817130,0.825206,0.809211,0.818902,0.898419,0.816596,0.825789,0.807606,0.818616,25.195124


In [18]:
def download_artifact_csv(run_id: str, contains: str) -> pd.DataFrame:
    arts = client.list_artifacts(run_id)
    # récursif simple
    stack = arts[:]
    target_path = None
    while stack:
        a = stack.pop()
        if a.is_dir:
            stack.extend(client.list_artifacts(run_id, a.path))
        else:
            if contains in a.path and a.path.endswith(".csv"):
                target_path = a.path
                break
    if target_path is None:
        raise FileNotFoundError(f"No artifact containing '{contains}' found for run {run_id}")

    local = mlflow.artifacts.download_artifacts(run_id=run_id, artifact_path=target_path)
    return pd.read_csv(local), target_path


In [19]:
def find_run_id_by_name(substring: str) -> str:
    sub = df_runs[df_runs["run_name"].str.contains(substring, case=False, na=False)]
    if sub.empty:
        raise ValueError(f"No run found with name containing '{substring}'")
    # prend le meilleur val_auc parmi ceux-là
    best = sub.sort_values("val_auc", ascending=False).iloc[0]
    run_name = best["run_name"]
    # retrouver le run_id correspondant
    for r in runs:
        if r.data.tags.get("mlflow.runName","") == run_name:
            return r.info.run_id
    raise RuntimeError("run_id not found back")

run_id_bert = find_run_id_by_name("bert_")
run_id_lstm = find_run_id_by_name("dl_lstm")
# baseline : adapte "baseline" à ton vrai nom de run
run_id_base = find_run_id_by_name("baseline")

run_id_base, run_id_lstm, run_id_bert


('40c53c9d38274903a1f77e6bf0213fac',
 'c3f0c21ac8e846ec9ca2e182d3b16584',
 '9e035c3264b9433f9058f9f4b6bb51cd')

In [20]:
def show_fp_fn(run_id: str, prefix_hint: str = ""):
    fp_df, fp_path = download_artifact_csv(run_id, "val_errors_false_positives")
    fn_df, fn_path = download_artifact_csv(run_id, "val_errors_false_negatives")

    print("FP artifact:", fp_path)
    print("FN artifact:", fn_path)

    # tri “confiance” : FP => proba haute ; FN => proba basse
    fp_df = fp_df.sort_values("proba_negative", ascending=False).head(15)
    fn_df = fn_df.sort_values("proba_negative", ascending=True).head(15)

    display(fp_df[["proba_negative","text"]])
    display(fn_df[["proba_negative","text"]])

print("=== BASELINE ===")
show_fp_fn(run_id_base)

print("=== LSTM ===")
show_fp_fn(run_id_lstm)

print("=== BERT ===")
show_fp_fn(run_id_bert)


/home/aurelien/formation_openclassrooms/projet_7/tweet_reaction_prediction/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


=== BASELINE ===


FP artifact: val_errors_false_positives.csv
FN artifact: val_errors_false_negatives.csv


,proba_negative,text
0,0.999770,i miss my stephie poo
1,0.999477,i wish
2,0.999477,wish hbd
3,0.999477,wish hbd
4,0.999477,i wish
5,0.999344,awww miley why so sad ur making me sad now i l...
6,0.999309,kristennnnnaaaaaaaaaa stop itaaaaaaa miss youu...
7,0.999086,lost outofthecloset
8,0.999060,its almost supper here im not hungery not feel...
9,0.998677,i feel like crying but i don t wanna cry get it


,proba_negative,text
0,0.000697,thanks federico
1,0.000697,thanks
2,0.000697,thanks
3,0.000697,thanks kacie
4,0.000697,thanks
5,0.000697,thanks
6,0.000697,thanks
7,0.001325,awesome
8,0.001413,hello renesme ur cute
9,0.001821,haha thanx


...

,proba_negative,text
0,0.002806,cool thank you thank you
1,0.002927,thanks enjoy your party
2,0.003661,yup thanks for the recommendation will check i...
3,0.005221,movie night with my sister and the pups so exc...
4,0.005230,croissants and milk and honey my healthy break...
5,0.005446,absolutely welcome to my world
6,0.005506,good start to the family trip out with the slo...
7,0.005627,lunch stop and enjoying the brilliant hot suns...
8,0.005728,hello renesme ur cute
9,0.005771,i m sure he s proud of you


=== BERT ===


FP artifact: bert_distilbert-base-uncased_len96_freeze0_val_errors_false_positives.csv
FN artifact: bert_distilbert-base-uncased_len96_freeze0_val_errors_false_negatives.csv


,proba_negative,text
0,0.998279,got the worst headache like ever unhappy
1,0.998190,so sick so sick of being tired and oh so tired...
2,0.998139,my body is having a bad reaction to camp i thi...
3,0.998109,my throat feels so injured
4,0.998086,my insides have died
5,0.998050,more days of school feeling sick again
6,0.998030,has poisoned herself not feeling good
7,0.997869,poor chuck sad hungry and very very stinky
8,0.997854,head hurts cut my hairr
9,0.997842,damn i just lost a follower again


,proba_negative,text
0,0.002877,dude watching this show about cults its amazin...
1,0.003349,welcome all our new followers we will follow y...
2,0.003545,thanks enjoy your party
3,0.003555,that quote is amazing i almost have up on love...
4,0.003585,a fine frenzy almost lover i love it
5,0.003687,is excited to make an excel spreadsheet of her...
6,0.003711,cool thank you thank you
7,0.003717,thanks for letting me know hope you have a goo...
8,0.004077,amazing day martin selina and laura i love u z...
9,0.004526,on the phone with puget sound energy yay


In [21]:
sweep_bert, sweep_path = download_artifact_csv(run_id_bert, "val_threshold_sweep")
sweep_bert.head(10)


,threshold,accuracy,precision,recall,f1,auc
0,0.50,0.848309,0.822426,0.888445,0.854162,0.929326
1,0.55,0.850750,0.837635,0.870172,0.853593,0.929326
2,0.45,0.844207,0.807047,0.904719,0.853096,0.929326
3,0.60,0.851043,0.852871,0.848453,0.850656,0.929326
4,0.40,0.837938,0.789900,0.920789,0.850337,0.929326
5,0.35,0.829836,0.773397,0.933055,0.845757,0.929326
6,0.65,0.849270,0.868473,0.823211,0.845237,0.929326
7,0.30,0.819297,0.755106,0.945109,0.839491,0.929326
8,0.70,0.843887,0.884604,0.790953,0.835161,0.929326
9,0.25,0.807066,0.736576,0.956047,0.832083,0.929326


In [22]:
best = sweep_bert.sort_values("f1", ascending=False).iloc[0]
best


threshold    0.500000
accuracy     0.848309
precision    0.822426
recall       0.888445
f1           0.854162
auc          0.929326
Name: 0, dtype: float64